In [1]:
!conda info


     active environment : ml_venv
    active env location : C:\Users\Shan\.conda\envs\ml_venv
            shell level : 1
       user config file : C:\Users\Shan\.condarc
 populated config files : C:\ProgramData\anaconda3\.condarc
                          C:\Users\Shan\.condarc
          conda version : 24.9.2
    conda-build version : 24.9.0
         python version : 3.12.7.final.0
                 solver : libmamba (default)
       virtual packages : __archspec=1=skylake
                          __conda=24.9.2=0
                          __cuda=12.8=0
                          __win=0=0
       base environment : C:\ProgramData\anaconda3  (read only)
      conda av data dir : C:\ProgramData\anaconda3\etc\conda
  conda av metadata url : None
           channel URLs : https://repo.anaconda.com/pkgs/main/win-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/win-64
                          https://repo.an

In [2]:
!java -version

openjdk version "11.0.27" 2025-04-15
OpenJDK Runtime Environment Temurin-11.0.27+6 (build 11.0.27+6)
OpenJDK 64-Bit Server VM Temurin-11.0.27+6 (build 11.0.27+6, mixed mode)


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('LoanData').getOrCreate()

In [5]:
spark

In [6]:
df= spark.read.csv('loan_data.csv',header=True, inferSchema=True)
df.show()

+---+------------------+-----------------+------------------+--------------+------------------+---+--------+
|_c0|            income|     credit_score|       loan_amount|years_employed|    debt_to_income|age|approved|
+---+------------------+-----------------+------------------+--------------+------------------+---+--------+
|  0|57.450712295168486| 696.308877376582| 43.99355436586002|            27|21.223491386654434| 22|       1|
|  1| 47.92603548243223|745.4708320235065|39.246336829127685|            27|24.836012237442116| 18|       1|
|  2|59.715328071510385|580.0716213090429| 30.59630369920174|            18|20.943750087879337| 34|       0|
|  3| 72.84544784612038|678.1484618345286|23.530632222944263|            16| 24.32608764281806| 39|       1|
|  4| 46.48769937914996|617.4678715439087|36.982233136135896|            13|30.408692224813464| 51|       1|
|  5|46.487945645762295|625.6437308117652|33.934853854217494|            18|33.847510547633135| 62|       1|
|  6| 73.6881922326

In [9]:
df.describe()

DataFrame[summary: string, _c0: string, income: string, credit_score: string, loan_amount: string, years_employed: string, debt_to_income: string, age: string, approved: string]

In [10]:
df = df.drop('_c0')
df.show()

+------------------+-----------------+------------------+--------------+------------------+---+--------+
|            income|     credit_score|       loan_amount|years_employed|    debt_to_income|age|approved|
+------------------+-----------------+------------------+--------------+------------------+---+--------+
|57.450712295168486| 696.308877376582| 43.99355436586002|            27|21.223491386654434| 22|       1|
| 47.92603548243223|745.4708320235065|39.246336829127685|            27|24.836012237442116| 18|       1|
|59.715328071510385|580.0716213090429| 30.59630369920174|            18|20.943750087879337| 34|       0|
| 72.84544784612038|678.1484618345286|23.530632222944263|            16| 24.32608764281806| 39|       1|
| 46.48769937914996|617.4678715439087|36.982233136135896|            13|30.408692224813464| 51|       1|
|46.487945645762295|625.6437308117652|33.934853854217494|            18|33.847510547633135| 62|       1|
| 73.68819223261087|620.3803037880565| 38.9519322002773

In [13]:
total_rows = df.count()
total_rows

500

In [14]:
len(df.columns)

7

In [15]:
df.groupBy('approved').count().show()

+--------+-----+
|approved|count|
+--------+-----+
|       1|  297|
|       0|  203|
+--------+-----+



In [17]:
class_count = df.groupBy('approved').count()
class_count.show()

+--------+-----+
|approved|count|
+--------+-----+
|       1|  297|
|       0|  203|
+--------+-----+



In [19]:
class_count_df = class_count.withColumn('percentage',class_count['count']/total_rows*100)

In [20]:
class_count_df.show()

+--------+-----+----------+
|approved|count|percentage|
+--------+-----+----------+
|       1|  297|      59.4|
|       0|  203|      40.6|
+--------+-----+----------+



In [21]:
df.groupBy('approved').avg('age').show()

+--------+------------------+
|approved|          avg(age)|
+--------+------------------+
|       1|42.861952861952865|
|       0| 43.68472906403941|
+--------+------------------+



In [22]:
df.groupBy('approved').avg('years_employed').show()

+--------+-------------------+
|approved|avg(years_employed)|
+--------+-------------------+
|       1| 14.878787878787879|
|       0|  14.64039408866995|
+--------+-------------------+



####  find out how many applicants with loan amounts greater than their income were approved or not.

In [32]:
filtered_df=df.filter(df['income']<df['loan_amount'])
filtered_df.groupBy('approved').count().show()

+--------+-----+
|approved|count|
+--------+-----+
|       0|   76|
+--------+-----+



#### calculate the average age of applicants who are older than 20 years and whose loan amount exceeds their income.

In [35]:
filtered_df1= df.filter((df['loan_amount']>df['income'])&(df['age']>20))
filtered_df1.agg({'age':'avg'}).show()

+-----------------+
|         avg(age)|
+-----------------+
|45.21917808219178|
+-----------------+



# Model Building

In [36]:
df.columns

['income',
 'credit_score',
 'loan_amount',
 'years_employed',
 'debt_to_income',
 'age',
 'approved']

In [37]:
from pyspark.ml.feature import VectorAssembler

In [39]:
assembler = VectorAssembler(inputCols=['income',
 'credit_score',
 'loan_amount',
 'years_employed',
 'debt_to_income',
 'age'],
outputCol='features' )
assembler

VectorAssembler_f168dbf6e29a

In [40]:
df_with_features = assembler.transform(df)
df_with_features.show()

+------------------+-----------------+------------------+--------------+------------------+---+--------+--------------------+
|            income|     credit_score|       loan_amount|years_employed|    debt_to_income|age|approved|            features|
+------------------+-----------------+------------------+--------------+------------------+---+--------+--------------------+
|57.450712295168486| 696.308877376582| 43.99355436586002|            27|21.223491386654434| 22|       1|[57.4507122951684...|
| 47.92603548243223|745.4708320235065|39.246336829127685|            27|24.836012237442116| 18|       1|[47.9260354824322...|
|59.715328071510385|580.0716213090429| 30.59630369920174|            18|20.943750087879337| 34|       0|[59.7153280715103...|
| 72.84544784612038|678.1484618345286|23.530632222944263|            16| 24.32608764281806| 39|       1|[72.8454478461203...|
| 46.48769937914996|617.4678715439087|36.982233136135896|            13|30.408692224813464| 51|       1|[46.4876993791

In [41]:
df_with_features.show(truncate=False)

+------------------+-----------------+------------------+--------------+------------------+---+--------+--------------------------------------------------------------------------------------+
|income            |credit_score     |loan_amount       |years_employed|debt_to_income    |age|approved|features                                                                              |
+------------------+-----------------+------------------+--------------+------------------+---+--------+--------------------------------------------------------------------------------------+
|57.450712295168486|696.308877376582 |43.99355436586002 |27            |21.223491386654434|22 |1       |[57.450712295168486,696.308877376582,43.99355436586002,27.0,21.223491386654434,22.0]  |
|47.92603548243223 |745.4708320235065|39.246336829127685|27            |24.836012237442116|18 |1       |[47.92603548243223,745.4708320235065,39.246336829127685,27.0,24.836012237442116,18.0] |
|59.715328071510385|580.0716213090429|30

In [42]:
model_input_df = df_with_features.select('features','approved')
model_input_df.show()

+--------------------+--------+
|            features|approved|
+--------------------+--------+
|[57.4507122951684...|       1|
|[47.9260354824322...|       1|
|[59.7153280715103...|       0|
|[72.8454478461203...|       1|
|[46.4876993791499...|       1|
|[46.4879456457622...|       1|
|[73.6881922326108...|       0|
|[61.5115209372936...|       0|
|[42.9578842109757...|       1|
|[58.1384006537894...|       0|
|[43.0487346078130...|       0|
|[43.0140536964461...|       0|
|[53.6294340734905...|       1|
|[21.3007963301330...|       0|
|[24.1262325123045...|       0|
|[41.5656870613854...|       1|
|[34.8075331949836...|       1|
|[54.7137099889291...|       1|
|[36.3796388671818...|       1|
|[28.8154444799706...|       0|
+--------------------+--------+
only showing top 20 rows



In [48]:
train_df,test_df = model_input_df.randomSplit([0.7,0.3],seed=10)

In [49]:
train_df.show()

+--------------------+--------+
|            features|approved|
+--------------------+--------+
|[1.38098989896391...|       0|
|[15.4711825289662...|       0|
|[18.9883684994018...|       0|
|[19.4115173335984...|       0|
|[20.7186830071624...|       0|
|[21.3007963301330...|       0|
|[21.9910221111237...|       0|
|[23.6189077036532...|       0|
|[24.1262325123045...|       0|
|[24.4492634096726...|       0|
|[25.8092619321552...|       0|
|[25.8877514815815...|       0|
|[25.9033051961364...|       0|
|[26.0835851180844...|       1|
|[26.7400485340080...|       0|
|[27.2094505106898...|       0|
|[27.2721340670217...|       0|
|[27.2772916297120...|       1|
|[27.8221701444885...|       0|
|[28.0472757780182...|       0|
+--------------------+--------+
only showing top 20 rows



In [50]:
test_df.show()

+--------------------+--------+
|            features|approved|
+--------------------+--------+
|[10.7038234386538...|       0|
|[18.1415641353528...|       0|
|[19.6228612001358...|       0|
|[20.1864662809866...|       0|
|[20.6049481418033...|       0|
|[21.2184317705143...|       0|
|[23.5543976695589...|       0|
|[24.3029820636368...|       1|
|[26.9882874389656...|       0|
|[28.2787348775401...|       1|
|[28.6287772067981...|       0|
|[28.8154444799706...|       0|
|[29.2580040355349...|       0|
|[29.2984781267766...|       0|
|[31.7871708081840...|       1|
|[31.9955538941633...|       1|
|[32.0569006387899...|       1|
|[32.1304475419602...|       0|
|[33.7767799457840...|       1|
|[33.8338283310604...|       1|
+--------------------+--------+
only showing top 20 rows



In [47]:
from pyspark.ml.classification import LogisticRegression

## Logistic Regression

In [55]:
lr = LogisticRegression(featuresCol='features',labelCol='approved')
model_lr=lr.fit(train_df)
lr_train_pred = model_lr.transform(train_df)
lr_test_pred = model_lr.transform(test_df)

In [54]:
from pyspark.ml.classification import DecisionTreeClassifier

## Decision Tree

In [56]:
dt = DecisionTreeClassifier(featuresCol='features',labelCol='approved',maxDepth=3,seed=30)
dt_model = dt.fit(train_df)
dt_train_pred = model_lr.transform(train_df)
dt_test_pred = model_lr.transform(test_df)

In [57]:
from pyspark.ml.classification import RandomForestClassifier

## Random Forest

In [58]:
rf = RandomForestClassifier(featuresCol='features',labelCol='approved',maxDepth=3,numTrees=100)
model_rf = rf.fit(train_df)
rf_train_pred = model_rf.transform(train_df)
rf_test_pred = model_rf.transform(test_df)

## Model Evaluation

In [59]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [69]:
accuracy_evaluator = MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='approved',metricName='accuracy')
f1_score_evaluator = MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='approved',metricName='f1')
precision_evaluator = MulticlassClassificationEvaluator(predictionCol='prediction',labelCol='approved',metricName='precisionByLabel')
recall_evaluator = MulticlassClassificationEvaluator(predictionCol = 'prediction',labelCol='approved',metricName='recallByLabel')

def evaluation(model_evaluation_df):
    print('Accuracy = ',accuracy_evaluator.evaluate(model_evaluation_df))
    print('F1 score = ',f1_score_evaluator.evaluate(model_evaluation_df))
    print('Precision = ',precision_evaluator.evaluate(model_evaluation_df))
    print('Recall = ',recall_evaluator.evaluate(model_evaluation_df))
    model_evaluation_df.groupBy('approved','prediction').count().show()

## Logistic Regression Evaluation

In [70]:
print('Training Performance')
evaluation(lr_train_pred)

Training Performance
Accuracy =  0.7543859649122807
F1 score =  0.7515432098765431
Precision =  0.7058823529411765
Recall =  0.631578947368421
+--------+----------+-----+
|approved|prediction|count|
+--------+----------+-----+
|       1|       0.0|   35|
|       0|       0.0|   84|
|       1|       1.0|  174|
|       0|       1.0|   49|
+--------+----------+-----+



In [71]:
print('Testing Performance')
evaluation(lr_test_pred)

Testing Performance
Accuracy =  0.740506329113924
F1 score =  0.7370748122622615
Precision =  0.7457627118644068
Recall =  0.6285714285714286
+--------+----------+-----+
|approved|prediction|count|
+--------+----------+-----+
|       1|       0.0|   15|
|       0|       0.0|   44|
|       1|       1.0|   73|
|       0|       1.0|   26|
+--------+----------+-----+



## Decision Tree

In [72]:
print('Training Performance')
evaluation(dt_train_pred)

Training Performance
Accuracy =  0.7543859649122807
F1 score =  0.7515432098765431
Precision =  0.7058823529411765
Recall =  0.631578947368421
+--------+----------+-----+
|approved|prediction|count|
+--------+----------+-----+
|       1|       0.0|   35|
|       0|       0.0|   84|
|       1|       1.0|  174|
|       0|       1.0|   49|
+--------+----------+-----+



In [73]:
print('Testing Performance')
evaluation(dt_test_pred)

Testing Performance
Accuracy =  0.740506329113924
F1 score =  0.7370748122622615
Precision =  0.7457627118644068
Recall =  0.6285714285714286
+--------+----------+-----+
|approved|prediction|count|
+--------+----------+-----+
|       1|       0.0|   15|
|       0|       0.0|   44|
|       1|       1.0|   73|
|       0|       1.0|   26|
+--------+----------+-----+



## Random Forest

In [74]:
print('Training Performance')
evaluation(rf_train_pred)

Training Performance
Accuracy =  0.9444444444444444
F1 score =  0.9438549726050987
Precision =  0.975
Recall =  0.8796992481203008
+--------+----------+-----+
|approved|prediction|count|
+--------+----------+-----+
|       1|       0.0|    3|
|       0|       0.0|  117|
|       1|       1.0|  206|
|       0|       1.0|   16|
+--------+----------+-----+



In [75]:
print('Testing Performance')
evaluation(rf_test_pred)

Testing Performance
Accuracy =  0.9430379746835443
F1 score =  0.9428916382811898
Precision =  0.9552238805970149
Recall =  0.9142857142857143
+--------+----------+-----+
|approved|prediction|count|
+--------+----------+-----+
|       1|       0.0|    3|
|       0|       0.0|   64|
|       1|       1.0|   85|
|       0|       1.0|    6|
+--------+----------+-----+



In [76]:
!pip install mrjob

In [77]:
from mrjob.job import MRJob

class AvgApprovalAge(MRJob):
  def mapper(self, _, line):
    entries = line.split(',')
    if entries[1] == 'income':
      return
    age = int(entries[6])
    approved = int(entries[7])
    yield (approved, age)

  def reducer(self, key, values):
    values = list(values)
    yield (key, sum(values)/len(values))

if __name__ == "__main__":
  AvgApprovalAge.run()

usage: ipykernel_launcher.py [options] [input files]
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

C:\Users\Shan\.conda\envs\ml_venv\Lib\site-packages\IPython\core\interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [78]:
!python mr.py --mapper loan_data.csv

1	22
1	18
0	34
1	39
1	51
1	62
0	58
0	46
1	21
0	22
0	28
0	68
1	38
0	52
0	29
1	43
1	68
1	36
1	62
0	24
0	64
0	24
1	43
0	57
0	22
0	38
0	31
1	21
1	40
1	59
1	37
0	32
1	67
0	69
1	62
0	46
1	25
0	58
0	65
1	36
1	65
0	56
0	63
1	45
0	63
0	44
1	21
1	32
1	52
0	54
1	18
1	20
1	64
1	41
1	44
1	57
1	53
1	51
1	46
1	41
0	41
1	59
0	21
1	20
0	51
1	66
0	40
0	69
0	40
1	62
1	22
1	48
1	58
1	23
0	54
0	27
1	57
0	24
0	56
0	55
1	38
1	37
0	20
0	68
1	39
1	64
1	21
1	45
0	29
1	55
1	68
1	53
1	64
1	58
1	22
0	30
1	40
1	67
1	47
1	54
0	42
1	65
0	44
0	68
0	31
0	65
1	64
0	53
0	56
0	33
0	39
1	38
1	38
0	25
1	66
1	33
0	48
0	59
1	27
1	49
1	51
0	26
0	34
1	28
1	49
1	27
0	39
1	34
1	21
1	65
0	69
1	64
0	42
1	38
0	54
0	55
0	66
0	18
0	42
0	68
1	53
1	34
0	58
0	41
1	19
0	57
0	32
1	49
0	56
0	23
1	18
1	64
1	40
0	56
1	34
0	51
1	58
1	46
0	26
0	30
1	23
1	23
1	37
1	69
1	20
1	34
1	66
1	29
0	65
1	64
0	62
0	18
1	34
0	20
0	43
1	32
0	53
1	49
1	49
1	21
1	35
0	66
0	36
1	60
0	69
1	52
1	57
1	31
1	50
1	31
1	51
0	21
0	41
0	46
1	36
1	23
0	50
0	66
1	57
0	36
